In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense

import tensorflow as tf

import numpy as np

model = Sequential()
model.add(Dense(64, input_dim=53, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='relu'))

model.compile(loss='mse', optimizer='adam')

Using TensorFlow backend.


In [2]:
def vote(situation):
    return b'\x00'

In [3]:
import random

random.seed(1234)

def get_n_actions(situation):
    for i in range(1,9):
        if situation[-2*i] != 0xFF:
            return 8-i+1
    return 0

def find_max(input_vector, n_actions):
    max_score = 0
    max_action = 0
    for i in range(n_actions):
        input_vector[-1] = i
        score = model.predict(np.array([input_vector]))
        if score > max_score:
            max_score = score
            max_action = i
    input_vector[-1] = max_action
    return max_score

def play(input_vector, n_actions):
    if random.random() > 0.95:
        input_vector[-1] = random.randrange(n_actions)
    else:
        find_max(input_vector, n_actions)
    return input_vector[-1]

def situation_to_input_vector(situation):
    situation.append(0)
    return np.asarray(situation)

In [4]:
prev_state = None
last_action = None
memory_input = list()
memory_label = list()

In [5]:
def conv_bytes_to_ints(byte_array):
    return list(byte_array)

In [6]:
def produce_label(reward, state_new):
    max_score = find_max(state_new, get_n_actions(state_new))
    return reward + 0.5 * max_score

In [7]:
def complete_request(payload_size, connection):
    data = list()
    while len(data) < payload_size:
        data += connection.recv(payload_size-len(data))
    return data

In [8]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serv_addr = ("0.0.0.0", 8080)
sock.bind(serv_addr)
sock.listen(1)

In [ ]:
while True:
    connection, client_address = sock.accept()
    prev_state_action = None

    while True:
        cmd = connection.recv(4)
        
        if cmd.decode() == "VOTE":
            complete_request(36, connection)
            connection.send(b'\x00')
        
        elif cmd.decode() == "PLAY":
            situation = complete_request(52, connection)
            n_actions = get_n_actions(situation)
            assert n_actions, situation
            input_vector = situation_to_input_vector(situation)
            prev_state_action = input_vector
            last_action = play(input_vector, n_actions)
            connection.send(int(last_action).to_bytes(1,"big"))
        
        elif cmd.decode() == "RSLT":
            result = complete_request(53, connection)
            memory_input.append(prev_state_action)
            new_state = conv_bytes_to_ints(result[1:53])
            label = produce_label(result[0], situation_to_input_vector(new_state))
            memory_label.append(label)
            connection.send(b'\x00')
        
        else:
            break

In [11]:
model.fit(np.asarray(memory_input), np.asarray(memory_label), epochs=20)

Epoch 1/20
204272/204272 [==============================] - 17s 83us/step - loss: 171.7983
Epoch 2/20
204272/204272 [==============================] - 18s 88us/step - loss: 171.1465
Epoch 3/20
204272/204272 [==============================] - 18s 88us/step - loss: 170.5166
Epoch 4/20
204272/204272 [==============================] - 19s 92us/step - loss: 170.0820
Epoch 5/20
204272/204272 [==============================] - 18s 90us/step - loss: 169.7066
Epoch 6/20
204272/204272 [==============================] - 18s 89us/step - loss: 169.3257
Epoch 7/20
204272/204272 [==============================] - 18s 90us/step - loss: 168.8765
Epoch 8/20
204272/204272 [==============================] - 18s 89us/step - loss: 168.7113
Epoch 9/20
204272/204272 [==============================] - 17s 84us/step - loss: 168.4358
Epoch 10/20
204272/204272 [==============================] - 17s 85us/step - loss: 168.2437
Epoch 11/20
204272/204272 [==============================] - 18s 89us/step - loss: 168.06